# Function: Trim to Relevant Part - Case Type Specific

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, StructType, StructField

def trim_judgment(full_text):
    """
    Trims judicial text to save LLM token.

    If "查明" is found, it trims before this keyword and returns the rest. If not found, it trims away the first third
    of the text, returning the remainder. Additionally, it returns the trimming type and twenty characters around the
    trimming point for diagnosis.

    Returns:
    - trimmed_text (str): The result of the trimming operation.
    - trimmedType (str): The method of trimming ('查明' or 'last 2/3').
    - text_around_trimmed_point (str): Context around the point of trimming.

    """

    if full_text is None:
        return None, "ERROR", ""
    
    if not isinstance(full_text, str):
        return None, "ERROR", ""
    
    # Search for "查明" from the beginning
    index = full_text.find("查明")
    
    if index != -1:
        # If "查明" is found, trim the text before "查明"
        trimmed_text = full_text[index + 2:]  # +2 to exclude "查明" itself
        trimmedType = "查明"
        # Calculate the position to start capturing text around "查明" safely
        start_pos = max(0, index - 10)  # Start from 10 chars before "查明", if possible
        end_pos = min(len(full_text), index + 10 + 2)  # Capture up to 10 chars after "查明", safely
        text_around_trimmed_point = full_text[start_pos:end_pos]
    else:
        # If "查明" is not found, trim the first third of the text
        one_third_length = len(full_text) // 3
        trimmed_text = full_text[one_third_length:]
        trimmedType = "last 2/3"
        # Determine the break point and safely capture twenty characters around it
        start_pos = max(0, one_third_length - 10)  # Start from 10 chars before the break point, if possible
        end_pos = min(len(full_text), one_third_length + 10)  # Capture up to 10 chars after the break point, safely
        text_around_trimmed_point = full_text[start_pos:end_pos]
    
    return trimmed_text, trimmedType, text_around_trimmed_point

# Function: Post Request and fetch Claude Response

In [ ]:
import time
import requests
import json
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType

api_key = dbutils.secrets.get(scope = "OhMyGPTAPI", key = "OhMyGPTAPI1")

def trim_and_fetch_facts(judgment_text):
    
    trimmed_judgment_text, trimmedType, text_around_trimmed_point = trim_judgment(judgment_text)

    url = "https://cfcus02.opapi.win/v1/messages"
    payload = json.dumps({
        "model": "claude-3-haiku-20240307",
        "stream": False,
        "system": "user will give you a legal judgement in Chinese. Please extract the name and amount of drugs from the judgment and output it within a JSON object. Use JSON format with the keys 'drug_a', 'amount_a', and 'drug_b', 'amount_b'. Be concise, and reply strictly using this format, {'drug':, }. Do not add anything else to the reply. If you can't extract the kind of drug or amount. reply 'NA'. reply using Chinese.",
        "messages": [
            {"role": "user", "content": trimmed_judgment_text}, 
            {"role": "assistant", "content": "{"}
            ],
        "max_tokens": 4096
    })
    headers = {
        "Authorization": 'Bearer ' + api_key,
        'Content-Type': 'application/json'
    }

    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.post(url, headers=headers, data=payload, timeout=20)
            response.raise_for_status()
            parsed_response = json.loads(response.text)
            # Check if the expected keys/path exists
            if "content" in parsed_response and len(parsed_response["content"]) > 0 and "text" in parsed_response["content"][0]:
                response_text = parsed_response["content"][0]["text"]
            else:
                raise KeyError("Unexpected response structure")
            return response_text, trimmedType, text_around_trimmed_point
        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"Error on attempt {attempt+1}: {e}")
        time.sleep(2 ** attempt + 1)

    # Default values in case of failure
    return "Failed to fetch response from API", "Error", "Error"



In [ ]:
#API test block:
text_0 = "深圳市宝安区人民法院刑 事 判 决 书（2012）深宝法刑初字第2122号公诉机关深圳市宝安区人民检察院。被告人唐某，因涉嫌犯贩卖毒品罪，于2011年12月15日被抓获，次日被刑事拘留，2012年1月16日被逮捕。现押于深圳市宝安区看守所。深圳市宝安区人民检察院以深宝检公二诉（2O12）1040号起诉书指控被告人唐某犯贩卖毒品罪，于2012年3月30日向本院提起公诉。本院依法适用简易程序，公开开庭审理了本案。被告人唐某到庭参加诉讼。现已审理终结。经审理查明，2011年12月15日，吸毒人员胡某某致电被告人唐某称欲购买毒品，并约定在某溜冰场附近交易。当日20时许，被告人唐某携带毒品到达约定交易地点，交易完毕被当场抓获，并在被告人唐某身上查获疑似冰毒一包（0.25克，未检出常见毒品）、手机一部，在吸毒人员胡某某处缴获冰毒一包（0.13克，经鉴定检出甲基苯丙胺）上述事实，被告人唐某在庭审过程中亦无异议，并有相关证据予以证实，足以认定。本院认为，被告人唐某无视国家法律，贩卖毒品给他人，其行为已构成贩卖毒品罪。公诉机关指控的罪名成立。鉴于被告人唐某归案后认罪态度较好，有一定悔罪表现，本院酌情从轻处罚。综合考虑被告人的认罪态度、社会危害性，依据《中华人民共和国刑法》第三百四十七条第四款、第六十四条之规定，判决如下：一、被告人唐某犯贩卖毒品罪，判处有期徒刑七个月（刑期从判决执行之日起计算。判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2011年12月15日至2012年7月14日止），并处罚金人民币一千元（罚金自本判决生效之日起十日内缴纳）。二、扣押在案的毒品依法予以销毁。如不服本判决，可在接到判决书的第二日起十日内，通过本院或者直接向深圳市中级人民法院提出上诉，书面上诉的，应当提交上诉状正本一份，副本二份。代理审判员　王　　　　灿二〇一二年四月十日书　记　员　刘洪印（兼）书　记　员　张　 雪　 娜声明：本网站公布的裁判文书仅供阅读参考，正式文本以生效裁判文书正本为准，其他网站不得转载！ 更多数据：www.macrodatas.cn"

text_1 ="广东省汕尾市中级人民法院刑 事 判 决 书（2012）汕尾中法刑一初字第25号公诉机关广东省汕尾市人民检察院。被告人陈水荣，男，1970年7月15日出生，汉族，广东省陆丰市人，小学文化，打工，住陆丰市。因本案于2011年4月27日被羁押，次日被刑事拘留，同年5月30日被逮捕。2012年2月14日因非法持有枪支罪，被本院判处有期徒刑一年。现羁押于陆丰市看守所。辩护人方振宏，广东宏进律师事务所律师。汕尾市人民检察院以汕检刑诉字(2012)36号起诉书指控被告人陈水荣犯贩卖毒品罪一案，于2012年4月17日向本院提起公诉。本院依法组成合议庭，于2012年5月15日公开开庭审理了本案。汕尾市人民检察院指派检察员李杰出庭支持公诉，被告人陈水荣及其辩护人方振宏到庭参加诉讼，现已审理终结。汕尾市人民检察院指控，2011年4月12日，被告人陈水荣在惠来县览表村向吴镇洲以每公斤204000元的价格购买毒品冰毒960克，然后以原价转手卖给陆丰市甲西镇的林泽波，并伙同林泽波、“毛毛”、“阿西”、“宝龙”四人将该批冰毒送到东莞市长安镇“嘉柏酒店”的713号房，经在房间现场称量，然后由林泽波将该批冰毒以每克220元的价格卖给林某1等人。为证明上述事实，公诉机关当庭出示了物证、书证，宣读了证人证言、现场勘查笔录、鉴定结论及被告人供述等证据。汕尾市人民检察院认为,被告人陈水荣的行为触犯《中华人民共和国刑法》第三百四十七条第二款第（一）项之规定,应当以贩卖毒品罪追究其刑事责任。提请本院依法惩处。被告人陈水荣对起诉指控的犯罪事实提出异议，称林泽波向其借了68000元做废品生意，其没有贩卖毒品给林泽波，其不认识林某1，亦没有与林泽波等人携带毒品去东莞的“嘉柏酒店”贩卖给林某1。辩护人提出，指控被告人陈水荣贩卖冰毒960克的证据不足，本案没有毒品、毒资的实物，陈水荣否认有贩毒的事实，且其供述前后矛盾，与林某1的证言存在严重出入，证人李某的证言不是直接证据。请求对被告人陈水荣作出无罪的判决。经审理查明，2011年4月13日，被告人陈水荣为向陆丰市甲西镇的林泽波讨回之前林泽波所欠的毒资，在惠来县览表村向他人以每公斤204000元的价格，购买毒品冰毒960克，然后以原价转手卖给林泽波，并伙同林泽波、“毛毛”、“阿西”、“宝龙”四人将该批冰毒送到东莞市长安镇“嘉柏酒店”的2713号房，经林某1在房间现场称量，该批冰毒为960克，然后由林泽波将该批冰毒以每克220元的价格卖给林某1300克及贩卖给他人。林某1当场付现金人民币3.2万元及有部分存款的农行卡给林泽波，再由林泽波将钱和卡转交给陈水荣，并告知了陈水荣该农行卡的密码为198110。因被告人陈水荣被林泽波拖欠部分毒资，在向林泽波讨要时被林泽波殴打威胁，因而自备仿“六四”手枪一支、子弹两发伺机报复林泽波。2011年4月27日下午14时30分许，被告人陈水荣驾驶“粤Ｂ×××××”号黑色北京现代伊兰特小轿车途经陆丰市甲东镇洋美村与水口村交叉路口时被陆丰市公安局甲东边防派出所民警查获，当场从陈水荣身上搜出自制仿“六四”手枪一支、子弹两发、可疑毒品1.31克。经广东省汕尾市公安司法鉴定中心痕迹检验鉴定，陈水荣所持枪支为自制仿“六四”手枪，能正常击发。可疑毒品经检验，检出甲基苯丙胺成分。认定上述事实，有经庭审质证的如下证据证实：（一）物证、书证1．陆丰市公安局扣押物品、文件清单及相片，证实甲东边防派出所于2011年4月27日扣押陈水荣持有的仿制手枪一支、子弹两发、白色晶体一小包等物品。2.东莞长安嘉柏酒店出具的客房帐单，证实林镇坤于2011年4月份二次入住该酒店2713号房，具体为：4月10日14时04分登记入住，11日13时44分离开该酒店；4月13日22时25分登记入住，同月15日14时14分离开该酒店。3.户籍证明，证实被告人陈水荣于1970年7月15日出生于广东省陆丰市，住广东省陆丰市甲东镇洋美村东五直巷113号。4.手机通话记录清单，证明被告人陈水荣与林泽波、林某1、李某、林正等多人于2011年3月28日至同年4月29日有利用手机进行通话的情况。5.抓获经过，证实陆丰市公安局甲东边防派出所于2011年4月27日14时10分，接到线报，辖区一名涉枪、贩毒人员开车从洋美村向甲子方向行驶，该所干警在洋美村路口守候，于14时20分将驾驶黑色现代小车（车牌粤Ｂ×××××）经过该处的被告人陈水荣抓获，从陈的身上搜获手枪1支（内有子弹2发），白色结晶状物1包等物品。6.（2012）汕尾中法刑一初字第4号刑事判决书，证实被告人陈水荣因犯非法持有枪支罪，被本院判处有期徒刑一年。刑期执行至2012年4月26日止。7.看守所收押人员体检登记表，证实2011年4月28日，被告人陈水荣经医生检查，身体健康情况正常。（二）证人证言1.证人林某1的证言，我是经同村的林泽波介绍认识陈水荣。2011年4月12日左右的一天晚上，林泽波打电话给我，说他和陈水荣等人带了一条货（1公斤冰毒）来长安，约我在长安嘉柏酒店见面，因为我们平时在嘉柏酒店都是在713房谈事。林镇坤于当晚开了713房和813房。所以我就在当晚约11点多钟到达713房。我在房里等了约半个小时，林泽波、陈水荣、宝龙等约4人到达。林泽波叫宝龙把货拿出来称一下，我当时就从身上拿出事先准备好的电子秤把那件装在透明胶纸袋的冰毒称了一下，是960克，陈水荣也是在现场验看。我在房里拿了一点试吸了几下，觉得质量还可以，就问林泽波，每克是多少钱，林泽波说每克是220元。然后我就向林泽波买了这批货的300克，并现场付了人民币32000元给林泽波，然后我又拿了一张以我名字开户的农行卡（在东莞长安开户的）交给林泽波，并把密码198110（我的生日）告诉林泽波，并说卡里有2万多元。扣除卡里的钱后还欠林泽波3000多元。我拿了这300克冰毒后，就在房间里和林泽波、陈水荣等人坐谈，林泽波还打电话给二个下家，约他们来嘉柏酒店713房买货。过了一会儿，一名叫阿力的湖南人过来房间，向林泽波买了大约200至300克冰毒。我走进房间里躺时，听见一名我以前认识的阳江男子来向林泽波购买冰毒，但买多少克我不清楚。我拿了买的300克冰毒和我的马仔肖伟剑送到东莞长安上沙村的阿杰家里，以每克250元的价钱贩卖给阿杰，我从中赚取9000元。下半夜，我回到嘉柏酒店的房间，林镇坤已在房间里，他和林泽波、陈水荣三人在房里溜冰毒，我和他们一起溜了一会儿冰毒，林泽波拿了一小包冰毒约40多克拿给林镇坤，我就把欠林泽波的3000多元还清了。第二天我就离开酒店了。这次林泽波、陈水荣等人带来的960克冰毒是陈水荣贩卖给林泽波的。陈水荣从哪里来的我不知道。陈水荣来东莞主要是来收毒资的。粤Ｂ×××××北京现代小汽车是湖南籍男子阿力以30000元转卖给我，我开了10多天后，转手以55000元卖给林泽波，然后林泽波因为拖欠陈水荣的毒资，又以约6万多元给陈水荣抵毒资。我听林泽波说陈水荣到他家里讨钱，相互打架，被林镇坤知道后放了回去。经混杂辨认照片，林某1辨认出5号就是陈水荣，就是与林泽波一起送冰毒给他的人。2.证人李某的证言，2011年4月下旬的一天晚上约8时，陈水荣打电话给我，说客寮村的林泽波打电话给他，叫其到他家拿3万元，叫我作伴一起去讨钱，以防万一。陈水荣开了一辆北京现代小轿车到甲子镇载我。在车上，陈水荣说，2011年4月上旬的一天，其从览表村向“毒头家”拿了一条（指一公斤）货（冰毒）然后转手经林泽波到东莞长安镇贩卖给林某1、林镇坤等人。最后这次生意陈水荣被林泽波欠了10多万元，其中1万元是向陈水荣的家属借的。他开的这辆小车是林泽波等人以6.5万元向他抵毒资的，但现在林泽波还欠他10多万元。我们到客寮村时已是晚上约9时多，陈水荣进入林泽波的家，我在车上等，10多分钟后，我绕到林泽波家的后窗观看，发现陈水荣在林泽波家里被几人围着，其中林泽波手拿一把猎枪顶着水荣，另一个叫宝龙的手持一把长刀围着水荣，另两个不知姓名的男青年手拿着长柄关刀围着水荣，我见此现状，就趁他们没发现，马上溜进村里躲起来。我在早上约6时独自坐摩托车准备回家，半途陈水荣打电话给我，我们在甲子车站见面，陈水荣说昨晚差点被林泽波等人打死。幸好后来林泽波的尾叔林镇坤持枪把陈水荣解救出来。陈水荣被林泽波等人打后的第三天晚上约6时，陈水荣来到我家里，说他这10多万元被林泽波欠去，讨钱还被林泽波持枪持刀殴打威胁，觉得不服气，陈水荣问我能否帮他购到手枪以防身。刚好我的朋友林某2来我家里，林某2说他想办法打听一下，并和陈水荣互留了联系电话，第二天早上9时，陈水荣曾经带着那把仿“六四”手枪到我家里拿给我看，但具体怎么弄到的我不清楚。我和陈水荣向林泽波讨钱过后的二三天，林泽波发给陈水荣信息，称要砍掉陈水荣的一只手，里面写的“光头”是我。经混杂辨认相片，李某辨认出9号相片（林泽波）就是欠陈水荣毒款不还，并持枪威胁殴打陈水荣的人。3.证人林某2的证言，2011年4月下旬一天晚上，李某到我家里向我求救，说陈水荣在近期曾经和林泽波做冰毒生意，被林泽波拖欠了10多万元，在今晚他和陈水荣上来向林泽波讨钱时，陈水荣被绑架起来了。要我帮其出面解决。我听后就独自走到林泽波家里，见林泽波家里有一名男子（指陈水荣），被林泽波、宝龙以及另二个年轻人围着，我对林泽波说，这个人是我一个契兄弟的叔伯兄弟，林泽波说不会打他，并说他们把数理直就放人。我回到家打了一个电话给林泽波的尾叔林镇坤，说有一亲威被林泽波扣押并要打他，林镇坤说要过去骂林泽波并叫他放人。我回家告诉李某，李在我家待到天亮就离开了。二天后，我在李某家里坐时，陈水荣提到因被林泽波欠钱并欺负一事，说准备买把手枪防身，李某要我帮陈水荣购买，我就提供一熟悉黑枪门路的广西司机“小平”的电话号码给他们,叫他们自己去联系。我不清楚陈水荣是否有购买到枪支，我也没有借枪给他。那天，在李某家吃了晚饭，陈水荣提议到麒麟山庄开房坐一下。晚上8时许，我开一辆白色别克小轿车，陈水荣开着一辆黑色现代轿车，来到了麒麟山庄，陈水荣开了一套房，我们在房内喝茶，过了40分钟，我就开车回深圳，半途我又返回去拿了手机。到深圳时是晚上12点多。经混杂辨认相片，林某2辨认出陈水荣、林泽波、林镇坤。4.证人蔡某的证言，2011年4月25日上午，甲东镇洋美村的一男子和其同伴开了一辆白色小轿车和一辆黑色小轿车，来我的山庄开了222号房间，当时房租150元还未付。该洋美村人的姓名我不清楚，但他的丈人叫邱控方，在开房记录里登记的为洋美村人，并在后面注明是邱控方的女夫（因“婿”字我不会写，就用“夫”字代替）。他们二人当天进出好多趟，他们二人是在次日早上离开的。经混杂辨认相片，蔡某指出第5号（林某2）、第10号（陈水荣）就是2011年5月25日来麒麟山庄登记住宿的人。（三）现场勘查、搜查笔录证实2011年4月27日14时20分，陆丰市公安局甲东边防派出所民警在甲东镇洋美村路口设伏，于14时30分许，截获陈水荣驾驶的黑色北京现代轿车，从陈水荣身上搜出一把仿制式黑色手枪、子弹两发，一包白色晶状固体。现场勘查制作了现场图一张，拍摄了现场相片。（四）鉴定结论1．(汕)公（司）鉴（痕）字（2011）027号痕迹检验报告，证实被告人陈水荣所持有的枪状物经检验，枪状物具备枪支的机构，且各机构的机件（枪管、击针、击锺、扳机等）完整；且配发“六四”式制式子弹。鉴定意见：送检的枪状物是自制仿“六四”式手枪，能正常击发。2.(汕)公（司）鉴（化）字（2011）082号刑事化验检验报告，证实公安机关将被告人陈水荣所持有的可疑毒品送检。经检验，该晶状物1小包，净重1.31克。检出甲基苯丙胺成分。（五）被告人供述被告人陈水荣在侦查阶段的供述，我认识林泽波是从其马仔“阿炎”来的，林泽波得知我能从惠来览表村购买到冰毒后，就在2011年4月6日下午3时许，“阿炎”带着林泽波去我家找我，并问我有没有冰毒卖给林泽波，我说还存有一点，林泽波拿了800元给我，我家里刚好存有吴彪的一些冰毒，我拿了约3克多冰毒给林泽波。二天后的中午约12时，阿炎打电话给我说林泽波验货后满意，叫我帮其搞约700至800克冰毒，我以每公斤冰毒18.5万元的价格向览表村吴彪购买671克冰毒，以每公斤冰毒19万元的价格卖给林泽波。林泽波还我7.9万元，还欠我3.5万元。4月12日还是13日（我想不起来），林泽波又向我要两件货（指2公斤冰毒），我因上次还欠吴彪3.5万元，所以不敢向他要货，于是我向览表村的吴镇州要两件货，但吴镇州称只有一件货。并说每件货要20.4万元，我为了向林泽波讨回上次的那3.5万元，这次的生意就不想挣他的钱，以每件20.4万元的价钱报给林泽波。我拿着一件货到北门庵附近和“阿炎”碰头，然后和“阿炎”带着这件货一林泽波家里，林称后只有960克。林泽波叫马仔“毛毛”开车载“阿西”，把一公斤的冰毒送到东莞，同时林泽波叫我与“宝龙”跟他开车尾随“毛毛”的车到东莞长安镇嘉柏酒店713房（林镇坤在当天晚上11时登记的），与林某1、林镇坤接头。林泽波将货交给林某1后就在该房秤量也是960克，当晚林泽波当着林某1和“阿西”的面拿了3.2万元现金还我，然后林某1还递了一张农行信用卡给林泽波，说卡里有6万元。林泽波就把卡交给我，并把密码报给我，我记得是198110。上午10点多钟，我就坐着林泽波的小车回家了，在路上，林泽波还从我那3.2万元里借了4000元，我回家后就把约3万元先还给吴彪，然后我那张林泽波给我的卡交给吴镇州，吴镇州取钱后说卡里只有2.16万元。之后林泽波又往卡里汇了部分钱还镇州，但林泽波还在有一天到我家里向我妻子借了1万元。最后经结算，林泽波欠我的货款和借款约18多万元。林泽波把一辆黑色奇瑞小车以1万元抵押给我，再把一辆黑色北京现代伊兰特小车（车牌粤Ｂ×××××）以6.3万元抵押给我，这辆车昨晚被扣押了。最后，林泽波结欠我11.54万元。那张农行卡我从吴镇州那取还林泽波了。4月22日左右，林泽波让我去他家先拿2万元，我在当晚8时许打电话叫李某和我一起去。李某没有进入林泽波家，我刚走进林泽波家，林泽波就拿出一把仿“五四”式手枪，他有两个马仔分别拿着来福枪顶在我的肋部，另一个叫宝龙的马仔拿着大砍刀威胁要砍我。林泽波用拳殴打我肩膀，并辱骂我，向我吐口水，说第一次货欠了100克，他还说当时交给我的农行卡里面有人民币50万元，被我取超过了。他要我将钱还给他，不然他就要把我杀了。后来是林泽波的三叔林镇坤拿了一把双管枪过去林泽波家里将我救出来的。我听说李某躲到第二天才坐摩托车回家的。我回家后一边是吴镇州找我讨要货款，一边是林泽波勒索我，并说我如果不还钱就要将我的手砍断。我找林正帮忙，他后来答应拿一把仿制手枪借我，让我去报复林泽波。2011年4月26日晚上12时许，林正在甲东镇新兴村麒麟山庄开了一间房间，并在房间里将那把仿制式手枪交给我。次日14时20分，我开着车带着借来的仿制手枪，打算去林泽波家里报复林泽波。从我身上查获的那一小包冰毒是吴镇洲另外送给我的。林某1是林泽波的堂兄弟，其手机号码为186××××5586，林泽波的手机号码为139××××3033，林镇坤的手机号码为151××××7060，宝龙的手机号码为134××××4884，李某的手机号码为136××××3499。被告人陈水荣在检察阶段和庭审中均翻供，称其并没有贩卖毒品给林泽波，没有与林泽波一起带冰毒去东莞。其不认识林某1。经混杂辨认相片，被告人陈水荣辨认其中一组相片中的1号为林某1，4号为林泽波，8号为吴镇州，9号为林镇坤。另一组相片中的2号为李某，5号为林正（指林某2）。关于被告人及辩护人所提的辩解辩护意见，经查：（1）证人林某1证实，其经林泽波的介绍认识陈水荣。在2011年4月12日左右的一天晚上11时多，林泽波、陈水荣、宝龙等约4人携带960克冰毒到东莞的嘉柏酒店713房，其以每克220元的价格向林泽波买了300克，并现场付了人民币3.2万元给林泽波，然后又拿了一张以其名字开户的农行卡交给林泽波，并把密码198110告诉林泽波，并说卡里有2万多元。扣除卡里的钱后还欠林泽波3000多元。并证实该960克冰毒是陈水荣贩卖给林泽波的。陈水荣被查扣的粤Ｂ×××××北京现代小汽车是阿力先卖给其，其转手卖给林泽波，林泽波因为拖欠陈水荣的毒资，又以约6万多元给陈水荣抵毒资的。其还证实听林泽波说陈水荣到他家里讨钱，相互打架，被林镇坤放了回去的事实经过，与被告人陈水荣在侦查阶段供述，其于4月12日还是13日，将960克冰毒贩卖给林泽波，并与林泽波及林的马仔“毛毛”、“阿西”等人把冰毒送到东莞长安镇嘉柏酒店713房与林某1、林镇坤接头。林某1在该房秤量冰毒重量为960克，当晚林泽波当着林某1和“阿西”的面还给其3.2万元现金，林某1还递了一张农行信用卡给林泽波。林泽波就把卡交给其，并报给其密码198110的经过能够相互印证。陈水荣及林某1均对对方进行了辨认。足以证明被告人陈水荣贩卖960克冰毒的事实属实。（2）认定交易毒品的地点在东莞长安嘉柏酒店2713号房，事实清楚，证据确实充分。东莞长安嘉柏酒店出具的客房帐单，证实林镇坤于2011年4月13日22时25分登记入住该酒店2713号房，15日14时14分离开该酒店，与被告人陈水荣及证人林某1所述交易毒品地点系在东莞长安嘉柏酒店713房，该房系林镇坤于案发当天（2011年4月12日左右）晚上11时登记的能相互印证。应当认定被告人陈水荣与林泽波等人携带毒品到东莞长安嘉柏酒店2713房的时间为2011年4月13日晚上。（3）手机通话记录清单，证明被告人陈水荣与林泽波、林某1、李某、林某2等多人于2011年3月28日至同年4月24日有利用手机进行通话的情况。其中被告人陈水荣（电话150××××2227）与林某1（电话186××××5586）有通过手机进行通话的记录，可以佐证被告人陈水荣是认识林某1的。并且可以证实被告人陈水荣所述通过手机与同案人联系毒品交易的事实属实。（4）证人林某1、李某均指证陈水荣所驾驶并被查扣的小汽车是林泽波以6.5万元向陈水荣抵毒资的，与被告人陈水荣对该情节的供述相互印证。证明被告人陈水荣与林泽波交易毒品的事实属实。（5）证人李某、林某2的证言证实了陈水荣在被林泽波拖欠毒资后，在向林泽波追讨时被林泽波殴打恐吓，而后又为报复林泽波而自备枪支的事实经过。证人林某1亦证明其听林泽波说陈水荣到他家里讨钱，相互打架，后被林镇坤放了回去的事实，上述证言与陈水荣对该情节的供述能相互印证。证明被告人陈水荣在与林泽波交易毒品后，被林泽波拖欠毒资，上门追讨时被林泽波殴打、恐吓，而后又自备枪支，意图报复的事实属实。上述证据，均经庭审举证质证，查证属实，并可排除逼供、诱供、串供的情形，来源合法，证据之间能相互印证，足以证实被告人陈水荣贩卖冰毒960克给林泽波，林泽波又将该批冰毒贩卖一部分给林某1的犯罪事实。被告人陈水荣与证人林某1虽然对林某1提供的农行卡里存款的数额（林某1说卡里有2万多元，陈水荣供述听说卡里有6万多元，经吴镇州取钱，卡里只有2.16万元）供述不一致，但可证实林泽波将林某1的农业银行卡交给陈水荣的事实属实。林某1供述，林泽波除将冰毒贩卖给其之外，还将剩余冰毒贩卖给他人，但这并不影响对陈水荣贩卖冰毒960克给林泽波的犯罪事实的认定。被告人陈水荣虽然在后来的供述中予以翻供，但其翻供没有相关证据证实，且与本案事实不符，其在侦查阶段所作的供述与在案的其他证据能相互印证。被告人陈水荣及其辩护人所提的辩解辩护意见，经查，理据不足，本院不予采纳。本院认为，被告人陈水荣无视国家法律，违反国家对毒品的管理制度，运输、贩卖毒品甲基苯丙胺960克，其行为已构成贩卖、运输毒品罪，依法应予惩处。公诉机关指控被告人贩卖毒品的犯罪事实清楚，证据确实充分，本院予以支持。但因被告人陈水荣对该宗毒品犯罪还实施了运输行为，故其罪名应定为贩卖、运输毒品罪，本院予以纠正。被告人及辩护人所提的辩解辩护意见，经查，理据不足，不能成立，本院不予采纳。被告人陈水荣前因犯非法持有枪支罪，被本院判处有期徒刑一年，刑罚执行完毕之前，发现陈水荣在判决宣告之前还有本罪没有判决，依法应当对被告人陈水荣实行数罪并罚。根据被告人陈水荣犯罪的事实、性质、情节和对于社会的危害程度，经本院审判委员会讨论，依照《中华人民共和国刑法》第三百四十七条第二款第（一）项、第五十七条第一款、第五十九条第一款、第六十四条、第六十九条、第七十条的规定，判决如下：一、被告人陈水荣犯贩卖、运输毒品罪，判处无期徒刑，剥夺政治权利终身，并处没收个人全部财产；与犯非法持有枪支罪，判处有期徒刑一年,数罪并罚；决定执行无期徒刑，剥夺政治权利终身，并处没收个人全部财产。二、被告人陈水荣被查获的毒品甲基苯丙胺1.31克，予以没收，由公安机关依法处理。如不服本判决，可在接到本判决书的第二日起十日内，通过本院或者直接向广东省高级人民法院提出上诉。书面上诉的，应当提交上诉状正本一份、副本二份。审判长　　骆金声审判员　　李　平审判员　　陈朝伟二〇一二年六月十日书记员　　曾向虹附相关法律条文：《中华人民共和国刑法》第三百四十七条走私、贩卖、运输、制造毒品，无论数量多少，都应当追究刑事责任，予以刑事处罚。走私、贩卖、运输、制造毒品，有下列情形之一的，处十五年有期徒刑、无期徒刑或者死刑，并处没收财产：（1）走私、贩卖、运输、制造鸦片一千克以上、海洛因或者甲基苯丙胺五十克以上或者其他毒品数量大的；（二）走私、贩卖、运输、制造毒品集团的首要分子；（三）武装掩护走私、贩卖、运输、制造毒品的；（四）以暴力抗拒检查、拘留、逮捕，情节严重的；（五）参与有组织的国际贩毒活动的。走私、贩卖、运输、制造鸦片二百克以上不满一千克、海洛因或者甲基苯丙胺十克以上不满五十克或者其他毒品数量较大的，处七年以上有期徒刑，并处罚金。走私、贩卖、运输、制造鸦片不满二百克、海洛因或者甲基苯丙胺不满十克或者其他少量毒品的，处三年以下有期徒刑、拘役或者管制，并处罚金；情节严重的，处三年以上七年以下有期徒刑，并处罚金。单位犯第二款、第三款、第四款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照各该款的规定处罚。利用、教唆未成年人走私、贩卖、运输、制造毒品，或者向未成年人出售毒品的，从重处罚。对多次走私、贩卖、运输、制造毒品，未经处理的，毒品数量累计计算。第五十七条对于被判处死刑、无期徒刑的犯罪分子，应当剥夺政治权利终身。在死刑缓期执行减为有期徒刑或者无期徒刑减为有期徒刑的时候，应当把附加剥夺政治权利的期限改为3年以上10年以下。第五十九条没收财产是没收犯罪分子个人所有财产的一部或者全部。没收全部财产的，应当对犯罪分子个人及其扶养的家属保留必需的生活费用。在判处没收财产的时候，不得没收属于犯罪分子家属所有或者应有的财产。第六十四条犯罪分子违法所得的一切财物，应当予以追缴或者责令退赔；对被害人的合法财产，应当及时返还;违禁品和供犯罪所用的本人财物，应当予以没收。没收的财物和罚金，一律上缴国库，不得挪用和自行处理。第六十九条判决宣告以前一人犯数罪的，除判处死刑和无期徒刑的以外，应当在总和刑期以下、数刑中最高刑期以上，酌情决定执行的刑期，但是管制最高不能超过三年，拘役最高不能超过一年，有期徒刑最高不能超过二十年。如果数罪中有判处附加刑的，附加刑仍须执行。第七十条判决宣告以后，刑罚执行完毕以前，发现被判刑的犯罪分子在判决宣告以前还有其他罪没有判决的，应当对新发现的罪作出判决，把前后两个判决所判处的刑罚，依照本法第六十九条的规定，决定执行的刑罚。已经执行的刑期，应当计算在新判决决定的刑期以内。 搜索“马 克 数 据 网”"

text_2 = "浙江省瑞安市人民法院刑 事 判 决 书（2012）温瑞刑初字第2151号公诉机关浙江省瑞安市人民检察院。被告人孙某。曾因吸毒毒品于1997年5月30日被瑞安市公安局行政拘留十日；因吸食毒品于2000年3月22日被温州市劳动教养管理委员会决定劳动教养二年；因吸食毒品于2002年2月10日被温州市劳动教养管理委员会决定劳动教养二年六个月；因吸食毒品于2006年5月11日被温州市劳动教养管理委员会决定劳动教养一年六个月；因吸食毒品于2008年12月16日被瑞安市公安局决定强制隔离戒毒二年；因吸食毒品于2011年12月9日被瑞安市公安局行政拘留十五日，并被瑞安市公安局责令接受社区戒毒三年；因吸毒毒品于2012年8月22日被瑞安市公安局决定强制隔离戒毒二年（未执行）。现因涉嫌犯贩卖毒品罪，于2012年8月8日被刑事拘留，同月30日被逮捕。现羁押于瑞安市看守所。瑞安市人民检察院以瑞检刑诉（2012）2670号起诉书指控被告人孙某犯贩卖毒品罪，于2012年11月7日向本院提起公诉。本院于次日立案后，依法适用简易程序，实行独任审判，公开开庭审理了本案。被告人孙某到庭参加诉讼。现已审理终结。瑞安市人民检察院指控：2012年8月7日20时许，被告人孙某通过手机与蔡某联系毒品交易的数量、价格及交易地点后，指使李某（另案处理）到瑞安市塘下镇塘西村老人亭附近桥上，将两包毒品海洛因以200元价格贩毒给蔡某后被公安人员现场抓获。继而，被告人孙某在塘下镇朝阳路被公安人员抓获，并从其身上查获一包毒品海洛因。缴获的毒品经鉴定，重量共计为0.23克，均检出海洛因成份。上述事实，被告人在开庭审理过程中亦无异议，并有证人李某、蔡某的证言、物证手机和毒品海洛因的照片、扣押物品清单、毒品上交清单、中国移动通信客户详单、理化检验报告书、公安行政处罚决定书、责令社区戒毒决定书、劳动教养决定书、强制戒毒决定书、抓获经过、人口信息等证据证实，足以认定。本院认为，被告人孙某明知是毒品而结伙予以贩卖，其行为已触犯刑律，构成贩卖毒品罪。公诉机关指控的罪名成立。为严明国法，惩罚犯罪，维护社会管理秩序，根据被告人犯罪的事实、犯罪的性质、情节和对于社会的危害程度，依照《中华人民共和国刑法》第三百四十七条第一款、第四款、第二十五条第一款、第六十四条之规定，判决如下：一、被告人孙某犯贩卖毒品罪，判处有期徒刑六个月，并处罚金人民币1000元。罚金限判决生效后十日内缴纳。（刑期从判决执行之日起计算，判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2012年8月8日起至2013年2月7日止）。二、随案移送的作案工具手机一部，予以没收。如不服本判决，可在接到判决书的第二日起十日内，通过本院或直接向浙江省温州市中级人民法院提出上诉。书面上诉的，应交上诉状正本一份，副本二份。审判员　　吴婉如二〇一二年十一月十四日书记员　　薛远芳 更多数据：搜索“马克数据网”来源：www.macrodatas.cn"

texts = [text_0, text_1, text_2]

for text in texts:
    print(trim_and_fetch_facts(text)[0])

# Read Csv file, conduct data enrichment, and save csv file (not updated yet, see test run code)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("ExtractFacts").getOrCreate()

# Base path where the data is saved
base_path = "/mnt/processed_data_criminal_case_analysis"

# Define the set of Causes of Action to filter and sample
causes_of_action = {"drug_related"}

# Initialize a dictionary to hold the counts
cases_count = {}

# Iterate over each cause of action to read the saved DataFrames, filter, and count the cases
for cause in causes_of_action:
    path = f"{base_path}/{cause}/*/*.csv"
    
    # Read the saved data
    df = spark.read.csv(path, header=True, inferSchema=True)
    
    # Filter to get rows where TrialProcedure is '一审'
    df_filtered = df.filter(col("TrialProcedure").contains("一审"))
    
    # Apply the UDF to the DataFrame and extract results into separate columns
    df_processed = df_filtered.withColumn("processed", trim_and_fetch_facts(df_filtered.FullText))

    df_final = df_processed.select(
        "*",  # Keep existing columns
        col("processed.response_text").alias("response_text"),
        col("processed.trimmedType").alias("trimmedType"),
        col("processed.text_around_trimmed_point").alias("text_around_trimmed_point")
    ).drop("processed")  # Drop the 'processed' struct column
    
    # Write the modified DataFrame back to a CSV, preserving the original structure
    output_path = f"{base_path}/{cause}_DrugTypeAmount_March_15"
    df_final.write.csv(output_path, mode="overwrite", header=True)

    
    # Count the number of cases for the current cause of action and update the dictionary
    count = df_filtered.count()
    cases_count[cause] = count

# Print the counts for each cause of action
for cause, count in cases_count.items():
    print(f"{cause} modified case count: {count}")


## Test Run: Move the Data Enrichment out of Spark

In [ ]:
import os
import pandas as pd

def process_files(base_path, cause):
    global total_rows_count, failed_fetch_count, na_in_response_count
    # Define the output folder based on the base path and cause
    output_folder = f"{base_path}/{cause}_DrugTypeAmount_March_15"
    
    # Path pattern to match CSV files for the cause of action
    path_pattern = os.path.join(base_path, cause)
    
    # List to collect DataFrames (if needed)
    dfs = []

    # Using dbutils.fs.ls to list directories/files
    files = dbutils.fs.ls(path_pattern)

    for file in files:
        # Check if the item is a directory and iterate through its contents
        if file.isDir():
            sub_files = dbutils.fs.ls(file.path)
            for sub_file in sub_files:
                if sub_file.name.endswith(".csv"):
                    # Reading CSV file into DataFrame
                    file_path = sub_file.path.replace("dbfs:", "/dbfs")  # Convert to local file path if necessary
                    df = pd.read_csv(file_path, on_bad_lines='skip')

                    
                    # Filter rows where 'TrialProcedure' column contains '一审'
                    df_filtered = df[df['TrialProcedure'].str.contains('一审', na=False)]

                    # Apply 'trim_and_fetch_facts' function to 'FullText' column
                    enriched_data = df_filtered['FullText'].apply(trim_and_fetch_facts)

                    if df_filtered.empty:
                        print(f"No data after filtering for {sub_file.name}. Moving to the next file.")
                        continue

                    if not isinstance(enriched_data.iloc[0], tuple) or len(enriched_data.iloc[0]) != 3:
                        print(f"Data structure mismatch in file: {file_path}")
                        continue
                    
                    # Expanding the enriched_data into separate columns
                    df_expanded = pd.DataFrame(enriched_data.tolist(), columns=['ResponseText', 'TrimmedType', 'TextAroundTrimmedPoint'], index=df_filtered.index)
                    df_filtered = pd.concat([df_filtered, df_expanded], axis=1)

                    total_rows_count += len(df_filtered)
                    failed_fetch_count += len(df_filtered[df_filtered['ResponseText'] == 'Failed to fetch response from API'])
                    na_in_response_count += len(df_filtered[df_filtered['ResponseText'].str.contains("NA", na=False)])
                    
                    
                    # Construct the output path for the enriched CSV file
                    output_file_path = 'dbfs:'+ os.path.join(output_folder, f"{os.path.basename(sub_file.name)}")
                    
                    # Save the processed DataFrame to the new CSV file, ensuring the path is in "/dbfs" format for local IO
                    df_filtered.to_csv(output_file_path.replace("dbfs:", "/dbfs"), index=False)


base_path = "/mnt/processed_data_criminal_case_analysis"
causes_of_action = ["drug_related"]

for cause in causes_of_action:
    total_rows_count, failed_fetch_count, na_in_response_count = 0, 0, 0
    process_files(base_path, cause)

    # Print the counts for process failures.
    print(f"For {cause}, Total rows processed: {total_rows_count}")
    print(f"For {cause}, Failed API fetch count: {failed_fetch_count}")
    print(f"For {cause}, NA in response count: {na_in_response_count}")

